In [1]:
import os

In [2]:
%pwd

'e:\\Projects\\Steganalysis\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'e:\\Projects\\Steganalysis'

In [43]:
from dataclasses import dataclass
from pathlib import Path
from typing import Tuple, List

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: Tuple[int, int, int]
    params_include_top: bool
    params_weights: str
    params_classes: int
    model_type: str          
    optimizer: str           
    loss_function: str       
    metrics: List[str]       
    batch_size: int          
    shuffle_data: bool       


In [6]:
from Steganalysis.constants import *
from Steganalysis.utils.common import read_yaml, create_directories

In [7]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [39]:
from box import ConfigBox
import yaml
from pathlib import Path
from Steganalysis.utils.common import read_yaml, create_directories
from Steganalysis.entity.config_entity import PrepareBaseModelConfig

CONFIG_FILE_PATH = 'E:/Projects/Steganalysis/config/config.yaml'
PARAMS_FILE_PATH = 'E:/Projects/Steganalysis/params.yaml'

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = ConfigBox(read_yaml(Path(config_filepath)))
        self.params = ConfigBox(read_yaml(Path(params_filepath)))
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        model_config = self.params.model
        training_config = self.params.training

        create_directories([self.config.prepare_base_model.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(self.config.prepare_base_model.root_dir),
            base_model_path=Path(self.config.prepare_base_model.base_model_path),
            updated_base_model_path=Path(self.config.prepare_base_model.updated_base_model_path),
            params_image_size=tuple(model_config.input_shape),
            params_include_top=model_config.include_top,
            params_weights=model_config.weights,
            params_classes=model_config.classes,
            model_type=model_config.model_type,
            optimizer=model_config.optimizer,
            loss_function=model_config.loss_function,
            metrics=model_config.metrics,
            batch_size=training_config.batch_size,
            shuffle_data=training_config.shuffle_data
        )

        return prepare_base_model_config


In [44]:
if __name__ == "__main__":
    config_manager = ConfigurationManager()
    prepare_base_model_config = config_manager.get_prepare_base_model_config()
    print("Model configuration:", prepare_base_model_config)
    print("Image size:", prepare_base_model_config.params_image_size)


[2024-11-09 15:21:41,528: INFO: common: yaml file: E:\Projects\Steganalysis\config\config.yaml loaded successfully]
[2024-11-09 15:21:41,541: INFO: common: yaml file: E:\Projects\Steganalysis\params.yaml loaded successfully]
[2024-11-09 15:21:41,545: INFO: common: created directory at: artifacts]
[2024-11-09 15:21:41,547: INFO: common: created directory at: artifacts/prepare_base_model]
Model configuration: PrepareBaseModelConfig(root_dir=WindowsPath('artifacts/prepare_base_model'), base_model_path=WindowsPath('artifacts/prepare_base_model/base_model.keras'), updated_base_model_path=WindowsPath('artifacts/prepare_base_model/base_model_updated.keras'), params_image_size=(512, 512, 3), params_include_top=False, params_weights='imagenet', params_classes=1, model_type='EfficientNetB3', optimizer='adam', loss_function='binary_crossentropy', metrics=BoxList(['accuracy']), batch_size=32, shuffle_data=True)
Image size: (512, 512, 3)


In [47]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
import efficientnet.tfkeras as efn

AUTO = tf.data.experimental.AUTOTUNE

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        self.strategy = self._get_strategy()
        self.model = None
        self.train_ds = None
        self.val_ds = None
        self.test_ds = None

    def _get_strategy(self):
        """Detect TPU or GPU strategy for distributed training."""
        try:
            tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
            print(f'Running on TPU {tpu.master()}')
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            return tf.distribute.experimental.TPUStrategy(tpu)
        except ValueError:
            print("TPU not detected, using default strategy.")
            return tf.distribute.get_strategy()

    # Inside PrepareBaseModel class after model creation and compilation:
    def get_base_model(self):
        with self.strategy.scope():
            if self.config.model_type == "EfficientNetB3":
                self.model = tf.keras.Sequential([
                    efn.EfficientNetB3(
                        input_shape=tuple(self.config.params_image_size),
                        weights=self.config.params_weights,
                        include_top=self.config.params_include_top
                    ),
                    layers.GlobalAveragePooling2D(),
                    layers.Dense(1, activation="sigmoid")
                ])
            else:
                raise ValueError(f"Model type {self.config.model_type} is not supported.")
            
            self.compile_model()
            self.model.summary()
            
            # Save model here
            self.save_model(self.config.base_model_path)  # Save the base model file


    def compile_model(self):
        """Compiles the model with configurations from params.yaml."""
        self.model.compile(
            optimizer=tf.keras.optimizers.get(self.config.optimizer),
            loss=self.config.loss_function,
            metrics=self.config.metrics
        )

    def prepare_data(self):
        """Prepares balanced data paths and splits them into train, validation, and test sets."""
        
        def get_paths(directory, start_index, end_index):
            """Helper function to get image paths from a directory within a range."""
            return [os.path.join(directory, filename) for filename in sorted(os.listdir(directory))[start_index:end_index]]

        # Define paths for each class
        cover_paths = np.array(get_paths('/kaggle/input/alaska2-image-steganalysis/Cover', 0, 75000))
        jmipod_paths = get_paths('/kaggle/input/alaska2-image-steganalysis/JMiPOD', 0, 25000)
        juniward_paths = get_paths('/kaggle/input/alaska2-image-steganalysis/JUNIWARD', 25000, 50000)
        uerd_paths = get_paths('/kaggle/input/alaska2-image-steganalysis/UERD', 50000, 75000)

        # Labels: Cover = 0, Stego = 1
        cover_labels = np.array([0] * len(cover_paths))
        stego_paths = np.array(jmipod_paths + juniward_paths + uerd_paths)
        stego_labels = np.array([1] * len(stego_paths))

        # Combine and shuffle
        img_paths = np.concatenate((cover_paths, stego_paths), axis=None)
        img_labels = np.concatenate((cover_labels, stego_labels), axis=None)

        # Split into train, test, validation sets
        X_train_paths, X_test_paths, y_train, y_test = train_test_split(img_paths, img_labels, test_size=0.15, shuffle=self.config.shuffle_data)
        X_train_paths, X_val_paths, y_train, y_val = train_test_split(X_train_paths, y_train, test_size=0.15, shuffle=self.config.shuffle_data)

        self.train_ds = self.build_dataset(X_train_paths, y_train)
        self.test_ds = self.build_dataset(X_test_paths, y_test)
        self.val_ds = self.build_dataset(X_val_paths, y_val)

    def data_augment(self, image, label):
        """Applies data augmentation."""
        image = tf.image.random_flip_left_right(image)
        return image, label  

    def decode_img(self, path, label):
        """Reads and decodes image from path, resizes, and normalizes."""
        bits = tf.io.read_file(path)
        image = tf.image.decode_jpeg(bits, channels=3)
        image = tf.image.resize(image, self.config.input_shape[:2]) / 255.0
        return image, label

    def build_dataset(self, X, y):
        """Builds a tf.data.Dataset for training, validation, or testing."""
        dataset = tf.data.Dataset.from_tensor_slices((X, y))
        dataset = dataset.map(self.decode_img, num_parallel_calls=AUTO)
        dataset = dataset.map(self.data_augment, num_parallel_calls=AUTO)
        dataset = dataset.batch(self.config.batch_size).prefetch(AUTO)
        return dataset

    def save_model(self, path):
        """Saves the model to the specified path."""
        self.model.save(path)
        print(f"Model saved at {path}")


In [48]:
config_manager = ConfigurationManager()
prepare_base_model_config = config_manager.get_prepare_base_model_config()
prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)

# Generate and print the model summary
prepare_base_model.get_base_model()


[2024-11-09 15:57:50,108: INFO: common: yaml file: E:\Projects\Steganalysis\config\config.yaml loaded successfully]
[2024-11-09 15:57:50,116: INFO: common: yaml file: E:\Projects\Steganalysis\params.yaml loaded successfully]
[2024-11-09 15:57:50,123: INFO: common: created directory at: artifacts]
[2024-11-09 15:57:50,123: INFO: common: created directory at: artifacts/prepare_base_model]


TPU not detected, using default strategy.


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnet-b3 (Functional)    │ (None, 16, 16, 1536)   │    10,783,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1536)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │         1,537 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,785,065 (41.14 MB)

 Trainable params: 10,697,769 (40.81 MB)

 Non-trainable params: 87,296 (341.00 KB)

Model saved at artifacts\prepare_base_model\base_model.keras
